## Data Cleaning and EDA

In [18]:
#Batch Imports
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
sns.set(style="ticks")
seed=42
import pickle

In [19]:
# Import Datasets
X_train = pd.read_csv('./data/train_tanzania.csv')
y_train = pd.read_csv('./data/train_label_tanzania.csv')
X_test = pd.read_csv('./data/test_set_tanzania.csv')


In [20]:
X_train.head(5)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [21]:
X_train.shape,y_train.shape

((59400, 40), (59400, 2))

In [22]:
y_train.head(5)

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


### For streamlined EDA, both independent variable sets were combined
#### randomized shuffling will be conducted later when re splitting fo TTS

In [23]:
df_X=pd.merge(X_train,y_train, on='id')

In [24]:
df = df_X.append(X_test, ignore_index = True)

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [25]:
print("Dataset has {} rows, {} columns.".format(*df.shape))

Dataset has 74250 rows, 41 columns.


In [26]:
def eda(dataframe): #Credit: Ritika Bhasker, DSI alum
    print ("missing values \n", dataframe.isnull().sum())
    print("")
    print ("dataframe types \n", dataframe.dtypes)
    print("")
    print ("dataframe unique vals \n", dataframe.nunique())
    
eda(df)

missing values 
 amount_tsh                   0
basin                        0
construction_year            0
date_recorded                0
district_code                0
extraction_type              0
extraction_type_class        0
extraction_type_group        0
funder                    4504
gps_height                   0
id                           0
installer                 4532
latitude                     0
lga                          0
longitude                    0
management                   0
management_group             0
num_private                  0
payment                      0
payment_type                 0
permit                    3793
population                   0
public_meeting            4155
quality_group                0
quantity                     0
quantity_group               0
recorded_by                  0
region                       0
region_code                  0
scheme_management         4846
scheme_name              35258
source                

In [27]:
# seperate features into numeric and categorical 
def identify_features():
    obs = df.select_dtypes(include=['object']).columns

    num = df.select_dtypes(include=[np.number]).columns

    print('There are {} categorical features'.format(len(obs)))
    print(obs)

    print('\n There are {} numerical features'.format(len(num)))
    return(obs,num)

In [28]:
cat,num = identify_features()

There are 31 categorical features
Index(['basin', 'date_recorded', 'extraction_type', 'extraction_type_class',
       'extraction_type_group', 'funder', 'installer', 'lga', 'management',
       'management_group', 'payment', 'payment_type', 'permit',
       'public_meeting', 'quality_group', 'quantity', 'quantity_group',
       'recorded_by', 'region', 'scheme_management', 'scheme_name', 'source',
       'source_class', 'source_type', 'status_group', 'subvillage', 'ward',
       'water_quality', 'waterpoint_type', 'waterpoint_type_group',
       'wpt_name'],
      dtype='object')

 There are 10 numerical features


In [29]:
cat

Index(['basin', 'date_recorded', 'extraction_type', 'extraction_type_class',
       'extraction_type_group', 'funder', 'installer', 'lga', 'management',
       'management_group', 'payment', 'payment_type', 'permit',
       'public_meeting', 'quality_group', 'quantity', 'quantity_group',
       'recorded_by', 'region', 'scheme_management', 'scheme_name', 'source',
       'source_class', 'source_type', 'status_group', 'subvillage', 'ward',
       'water_quality', 'waterpoint_type', 'waterpoint_type_group',
       'wpt_name'],
      dtype='object')

In [30]:
# visually identify the duplicated columns
df[cat].sample(n=12, random_state=42)

,basin,date_recorded,extraction_type,extraction_type_class,extraction_type_group,funder,installer,lga,management,management_group,payment,payment_type,permit,public_meeting,quality_group,quantity,quantity_group,recorded_by,region,scheme_management,scheme_name,source,source_class,source_type,status_group,subvillage,ward,water_quality,waterpoint_type,waterpoint_type_group,wpt_name
66920,Lake Victoria,2013-01-26,afridev,handpump,afridev,Marafip,DWE,Bunda,wug,user-group,pay when scheme fails,on failure,False,True,good,enough,enough,GeoData Consultants Ltd,Mara,WUG,NaN,shallow well,groundwater,shallow well,NaN,Mwilindi,Kisorya,soft,hand pump,hand pump,Kwa Manjar
7873,Pangani,2011-03-05,gravity,gravity,gravity,Government Of Tanzania,DWE,Lushoto,vwc,user-group,never pay,never pay,True,True,good,enough,enough,GeoData Consultants Ltd,Tanga,VWC,Shagai streem,spring,groundwater,spring,functional,Mgwashi,Mbaramo,soft,communal standpipe,communal standpipe,Kwa Mzee Timoni
8667,Rufiji,2011-04-12,gravity,gravity,gravity,Danida,DANIDA,Mbeya Rural,wug,user-group,pay when scheme fails,on failure,False,False,good,enough,enough,GeoData Consultants Ltd,Mbeya,VWC,NaN,river,surface,river/lake,functional,Itete,Ilungu,soft,communal standpipe,communal standpipe,Kwa Mvulula
71951,Ruvuma / Southern Coast,2013-01-19,mono,motorpump,mono,Government Of Tanzania,District Water Department,Nachingwea,vwc,user-group,pay per bucket,per bucket,False,True,unknown,dry,dry,GeoData Consultants Ltd,Lindi,VWC,Chimbendenga,machine dbh,groundwater,borehole,NaN,Nyerere Road,Mbondo,unknown,communal standpipe multiple,communal standpipe,Shuleni
29926,Lake Nyasa,2011-03-27,nira/tanira,handpump,nira/tanira,NaN,NaN,Ileje,vwc,user-group,unknown,unknown,False,True,good,seasonal,seasonal,GeoData Consultants Ltd,Mbeya,VWC,NaN,shallow well,groundwater,shallow well,functional,Isoka,Mbebe,soft,hand pump,hand pump,Kwaibhare
19838,Rufiji,2011-04-02,gravity,gravity,gravity,Village Council,VILLAGE COUNCIL,Mbeya Rural,wug,user-group,pay monthly,monthly,True,True,good,enough,enough,GeoData Consultants Ltd,Mbeya,VWC,NaN,river,surface,river/lake,functional,Simambwe,Igale,soft,communal standpipe,communal standpipe,Daima Sekele
59897,Lake Nyasa,2013-02-14,swn 80,handpump,swn 80,Danida,DANIDA,Namtumbo,wug,user-group,pay monthly,monthly,False,True,good,enough,enough,GeoData Consultants Ltd,Ruvuma,WUG,NaN,shallow well,groundwater,shallow well,NaN,Ngongoma,Kitanda,soft,hand pump,hand pump,Kwa Kauti
69927,Lake Nyasa,2011-04-08,other,other,other,Government Of Tanzania,Village Council,Mbeya Rural,parastatal,parastatal,pay when scheme fails,on failure,False,False,good,seasonal,seasonal,GeoData Consultants Ltd,Mbeya,Parastatal,NaN,rainwater harvesting,surface,rainwater harvesting,NaN,Mafune,Ilembo,soft,other,other,Kituo Cha Afya
9082,Rufiji,2011-10-04,gravity,gravity,gravity,Unicef,DWE,Njombe,wua,user-group,pay monthly,monthly,True,True,good,insufficient,insufficient,GeoData Consultants Ltd,Iringa,WUA,wanging'ombe water supply s,river,surface,river/lake,functional needs repair,Itula,Usuka,soft,communal standpipe,communal standpipe,Kwa Mbugi
60312,Internal,2013-02-19,nira/tanira,handpump,nira/tanira,NaN,NaN,Singida Rural,vwc,user-group,pay when scheme fails,on failure,NaN,True,good,insufficient,insufficient,GeoData Consultants Ltd,Singida,VWC,NaN,shallow well,groundwater,shallow well,NaN,Mampando A,Ntuntu,soft,hand pump,hand pump,Yusuphu Athum


In [31]:
df.isnull().sum().sort_values(ascending=False).head(10)

scheme_name          35258
status_group         14850
scheme_management     4846
installer             4532
funder                4504
public_meeting        4155
permit                3793
subvillage             470
district_code            0
extraction_type          0
dtype: int64

In [32]:
# drop excess features
df.drop(['scheme_name','source','payment','quantity',
         'waterpoint_type','scheme_management','public_meeting',
         'permit','extraction_type','region_code'],axis=1, inplace=True)



In [34]:
#drop rows with any null values in these columns 
df=df.dropna(subset=['funder','installer','subvillage'], how='any')


In [35]:
df.isnull().sum()

amount_tsh                   0
basin                        0
construction_year            0
date_recorded                0
district_code                0
extraction_type_class        0
extraction_type_group        0
funder                       0
gps_height                   0
id                           0
installer                    0
latitude                     0
lga                          0
longitude                    0
management                   0
management_group             0
num_private                  0
payment_type                 0
permit                     799
population                   0
quality_group                0
quantity_group               0
recorded_by                  0
region                       0
region_code                  0
source_class                 0
source_type                  0
status_group             13862
subvillage                   0
ward                         0
water_quality                0
waterpoint_type_group        0
wpt_name

In [ ]:
df.shape

### Numerical Data 

In [36]:
df[num].describe()

,amount_tsh,construction_year,district_code,gps_height,id,latitude,longitude,num_private,population,region_code
count,69183.000000,69183.000000,69183.000000,69183.000000,69183.000000,6.918300e+04,69183.000000,69183.000000,69183.000000,69183.000000
mean,331.272965,1345.086264,5.643858,679.693595,37138.483833,-5.618044e+00,34.040217,0.492361,188.831360,15.493430
std,2685.105337,936.494643,9.643272,691.269334,21434.201096,2.960383e+00,6.764853,11.933675,479.438286,17.922556
min,0.000000,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,2.000000,0.000000,18570.500000,-8.257811e+00,33.005367,0.000000,0.000000,5.000000
50%,0.000000,1988.000000,3.000000,401.000000,37141.000000,-4.900364e+00,35.028777,0.000000,40.000000,12.000000
75%,30.000000,2005.000000,5.000000,1317.000000,55696.500000,-3.300273e+00,37.295202,0.000000,234.000000,17.000000
max,250000.000000,2013.000000,80.000000,2777.000000,74249.000000,-2.000000e-08,40.345193,1776.000000,30500.000000,99.000000


In [37]:
df.region_code.value_counts(normalize=True).head(10)

11    0.095284
17    0.090658
3     0.079311
5     0.073212
18    0.058454
19    0.054204
16    0.050894
12    0.048336
2     0.047815
10    0.047743
Name: region_code, dtype: float64

In [38]:
df.district_code.value_counts(normalize=True).head(10)

1     0.216874
2     0.182877
3     0.171545
4     0.142419
5     0.074296
6     0.071130
7     0.048466
8     0.018487
30    0.018097
33    0.015307
Name: district_code, dtype: float64

In [39]:
#convert district codes to categorical variables
df['district_code'] = df['district_code'].astype(str)

In [43]:
#correlation analysis see if any of the features are duplicates and therefore candidates for removal
df.corr()

,amount_tsh,construction_year,gps_height,id,latitude,longitude,num_private,population
amount_tsh,1.000000,0.076826,0.087688,-0.009580,-0.068399,0.025502,0.005891,0.016414
construction_year,0.076826,1.000000,0.636677,-0.002586,-0.329048,0.418724,0.026742,0.258466
gps_height,0.087688,0.636677,1.000000,-0.003332,-0.088352,0.153083,0.007079,0.126425
id,-0.009580,-0.002586,-0.003332,1.000000,0.003569,-0.001597,-0.004652,-0.004808
latitude,-0.068399,-0.329048,-0.088352,0.003569,1.000000,-0.439390,0.004925,-0.038612
longitude,0.025502,0.418724,0.153083,-0.001597,-0.439390,1.000000,0.024955,0.088295
num_private,0.005891,0.026742,0.007079,-0.004652,0.004925,0.024955,1.000000,0.002941
population,0.016414,0.258466,0.126425,-0.004808,-0.038612,0.088295,0.002941,1.000000


In [45]:
df.isnull().sum()

amount_tsh                   0
basin                        0
construction_year            0
date_recorded                0
district_code                0
extraction_type_class        0
extraction_type_group        0
funder                       0
gps_height                   0
id                           0
installer                    0
latitude                     0
lga                          0
longitude                    0
management                   0
management_group             0
num_private                  0
payment_type                 0
population                   0
quality_group                0
quantity_group               0
recorded_by                  0
region                       0
source_class                 0
source_type                  0
status_group             13862
subvillage                   0
ward                         0
water_quality                0
waterpoint_type_group        0
wpt_name                     0
dtype: int64

### Seperate training and test sets 

In [46]:
df_test= df[df['status_group'].isnull()]
df_test.drop('status_group',axis=1,inplace=True)

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [ ]:
df_train=df.dropna(subset=['status_group'], how='any')

In [ ]:
df_test.shape,df_train.shape

In [ ]:
df_train.to_pickle('./data/train.pkl')
df_test.to_pickle('./data/test.pkl')